## Code for lstm network development. data preparation for lstm can be found in preprocess/seqdata.ipynb

In [24]:
import pandas as pd
import numpy as np
import time
import keras
from math import ceil
import time
import random
import glob,os
import matplotlib.pyplot as plt
import pickle
import pdb
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Input, Embedding, Masking, Dropout
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from sklearn.base import BaseEstimator
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib

### Load Data

In [ ]:
with open('/home/arash/ProjectVR/cleaneddata/seperateddata/inputseqtrain', 'rb') as f:
    inputseqtrain = pickle.load(f) 
with open('/home/arash/ProjectVR/cleaneddata/seperateddata/ytrain', 'rb') as f:
    ytrain = pickle.load(f) 
with open('/home/arash/ProjectVR/cleaneddata/seperateddata/inputseqTEST', 'rb') as f:
    inputseqTEST = pickle.load(f) 
with open('/home/arash/ProjectVR/cleaneddata/seperateddata/ytest', 'rb') as f:
    ytest = pickle.load(f) 

### Define Vanilla Model

In [17]:
class VanillaLSTM(BaseEstimator):
    
    def __init__(self, nodes=50, batch_size=32,
                 epochs=200, Llayers=1, features=5, steps_in=0, steps_out=0
                 ):

        self.nodes = nodes
        self.batch_size = batch_size
        self.epochs = epochs
        self.Llayers= Llayers
        self.features = features
        self.steps_in = steps_in
        self.steps_out = steps_out
        self.maxXY=654.7
        self.minXY=650.1


    def create_model(self):
        model = Sequential()
        model.add(Masking(mask_value=-0.01, input_shape=(self.steps_in, self.features)))
        for i in range(self.Llayers-1):
            model.add(LSTM(self.nodes, activation='relu', return_sequences=True))
        model.add(LSTM(self.nodes, activation='relu'))
        model.add(Dense(self.steps_out))
        model.compile(optimizer='adam', loss='mse')
        return model
    
    def fit(self,inputseq,y):
        start = time.time()
        X = inputseq[:,:,:self.features]
        self.model=self.create_model()
        self.model.fit(X, y, epochs = self.epochs, verbose = 0, batch_size = self.batch_size)
        end = time.time()
        print ("Finished Fitting Model. # of Epochs: %d\n Time Taken : %d secs"
               % (self.epochs,end - start))
        return self
    
    def predict(self, inputseq):
        X = inputseq[:,:,:self.features]
        
        return self.model.predict(X)
    
    def score(self,inputseq,y):
        
        X = inputseq[:,:,:self.features]
        ypred = self.predict(inputseq)
        yinv = np.zeros(y.shape)
        ypredinv = np.zeros(ypred.shape)
        for i in range(len(y)):
            pos = sum (n>=0 for n in y[i])                #position of the last actual value on y, not padded
            yinv[i][:pos]= y[i][:pos] * (self.maxXY-self.minXY) + self.minXY
            ypredinv[i][:pos] = ypred[i][:pos]* (self.maxXY-self.minXY) + self.minXY
            
        rmseScore = -sqrt(mean_squared_error(ypredinv, yinv))    #negative of RMSE

        return rmseScore
        

### fitting Vanilla LSTM, using grid search for hypperparameters

In [18]:
n_features = (inputseqtrain.shape[2]-1)
n_steps_in=inputseqtrain.shape[1]
n_steps_out=ytrain.shape[1]

VLmodel=VanillaLSTM(epochs=1, features=n_features,
                    steps_in=n_steps_in, steps_out=n_steps_out)

In [19]:
tuned_params = {"Llayers" : [2,3]}#{"nodes" : [10,50,100],"batch_size": [32,64,128] ,"Llayers" : [1,2,3]}

gs = GridSearchCV(VLmodel, tuned_params,cv = 8, refit= True, n_jobs=2)

In [20]:
grid_result=gs.fit(inputseqtrain,ytrain)

/home/arash/anaconda3/envs/gpusupport/lib/python3.6/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Finished Fitting Model. # of Epochs: 1
 Time Taken : 22 secs


In [ ]:
np.save('/home/arash/ProjectVR/models/GridResults/VLSTM400_8_50_32_L.npy',grid_result.cv_results_)      
#filename: ModelnameEpochs_CV_Nodes_Batch_changingparameters (L:LLayers)

In [21]:
bestVLSTM = grid_result.best_estimator_

In [ ]:
joblib.dump(bestVLSTM, '/home/arash/ProjectVR/models/trainedmodels/VLSTM1.pkl')

In [22]:
bestVLSTM.score(inputseqTEST,ytest)

-0.5619615507219169

In [ ]:
history = bestVLSTM.model.history.history
# plot history
plt.plot(history['loss'], label='train')
#pyplot.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()


## lstm model with aux variables

In [25]:
class AuxLSTM(BaseEstimator):
    
    def __init__(self, nodes=50, batch_size=32,
                 epochs=200, Dlayers=1, Llayers=1, features=5, steps_in=83, steps_out=147, drpout=0.5
                 ):

        self.nodes = nodes
        self.features = features
        self.steps_in = steps_in
        self.steps_out = steps_out
        self.epochs = epochs
        self.batch_size = batch_size
        self.Dlayers= Dlayers   #number of hidden dense layers
        self.Llayers = Llayers  #number of lstm layers
        self.maxXY=654.7
        self.minXY=650.1
        self.drpout=drpout


    def create_model(self):
        
        seq_input = Input(shape=(self.steps_in, self.features), dtype='float32', name='seq_input')
        mask = Masking(mask_value=-0.01)(seq_input)
        if self.Llayers==1:
            
            lstm_out = LSTM(self.nodes, activation='relu')(mask)
            
        else:
            lstm_out = LSTM(self.nodes,activation='relu',return_sequences=True)(mask)
            
            for i in range(self.Llayers-2):
                lstm_out = LSTM(self.nodes,activation='relu',return_sequences=True)(lstm_out)
                
            lstm_out = LSTM(self.nodes, activation='relu')(lstm_out)

        #output for lstm, corresponds to 0.2 of loss, used to smooth training and regularization:
        auxiliary_output = Dense(self.steps_out, activation='sigmoid' ,name='aux_output')(lstm_out)   

        auxiliary_input = Input(shape=(14,), name='aux_input')
        x = keras.layers.concatenate([lstm_out, auxiliary_input])

        # We stack a deep densely-connected network on top
        for i in range(self.Dlayers):
            x = Dense(self.nodes, activation='relu')(x)
            if self.drpout>0:
                x = Dropout(self.drpout)(x)

        # And finally we add the main logistic regression layer
        main_output = Dense(self.steps_out, activation='sigmoid', name='main_output')(x)

        modelaux = Model(inputs=[seq_input, auxiliary_input], outputs=[main_output, auxiliary_output])


        modelaux.compile(optimizer='adam', loss='mse',
                      loss_weights=[1., 0.2])
        return modelaux
    
    def fit(self,inputseq,y):
        start = time.time()
        X = inputseq[:,:,:self.features]
        aux=inputseq[:,:,self.features][:,:14]      
        self.model=self.create_model()
        self.model.fit([X,aux], [y,y], epochs = self.epochs, verbose = 0, batch_size = self.batch_size)
        end = time.time()
        print ("Finished Fitting AuxModel. # of Epochs: %d\n Time Taken : %d secs"
               % (self.epochs,end - start))
        return self
    
    def predict(self, inputseq):
        X = inputseq[:,:,:self.features]
        aux=inputseq[:,:,self.features][:,:14]
        
        return self.model.predict([X,aux])
    
    def score(self,inputseq,y):
        X = inputseq[:,:,:self.features]
        aux = inputseq[:,:,self.features][:,:14]    
        ypred = self.predict(inputseq)[0]        #auxlstm has two identical outputs, 1st in chosen
        yinv = np.zeros(y.shape)
        ypredinv = np.zeros(ypred.shape)
        for i in range(len(y)):
            pos = sum (n>=0 for n in y[i])                #position of the last actual value on y, not padded
            yinv[i][:pos]= y[i][:pos] * (self.maxXY-self.minXY) + self.minXY
            ypredinv[i][:pos] = ypred[i][:pos]* (self.maxXY-self.minXY) + self.minXY
            
        rmseScore = -sqrt(mean_squared_error(ypredinv, yinv))    #negative of RMSE
        return rmseScore
        

In [36]:
n_features = (inputseq.shape[2]-1)
n_steps_in=inputseq.shape[1]
n_steps_out=ytrain.shape[1]

Auxmodel=AuxLSTM(epochs=200, features=n_features,
                    steps_in=n_steps_in, steps_out=n_steps_out)
 

In [37]:
tuned_params = {'drpout': [0,0.2,0.5],"Dlayers" : [1,2,3], "Llayers" : [2,3]}

gsaux = GridSearchCV(Auxmodel, tuned_params,cv = 8, refit= True, n_jobs=-1)

In [ ]:
gridaux_result=gsaux.fit(inputseqtrain,ytrain)

In [ ]:
np.save('/home/arash/ProjectVR/models/GridResults/AuxLSTM200_8_DropDL.npy',gridaux_result.cv_results_)
bestauxLSTM = gridaux_result.best_estimator_

In [ ]:
joblib.dump(bestauxLSTM, '/home/arash/ProjectVR/models/trainedmodels/AuxLSTM3.pkl')
bestauxLSTM.score(inputseqTEST,ytest)

In [ ]:
history = bestauxLSTM.model.history.history
# plot history
plt.plot(history['loss'], label='train')
#pyplot.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()


In [ ]:
gridaux_result.cv_results_